In [1]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
# from keras.applications.vgg16 import VGG16
from tensorflow.python.client import device_lib

# from keras.utils import plot_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Dense, Flatten, LSTM, Activation, BatchNormalization
from tensorflow.keras.layers import Dropout, RepeatVector, TimeDistributed
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator

X = np.load('/content/drive/My Drive/emotion_data/X.npy')
y = np.load('/content/drive/My Drive/emotion_data/y.npy')
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

y = indices_to_one_hot(y.astype(int), 7)
# X = X/255
print(X.shape)
print(y.shape)
# print(np.unique(y))

(28709, 48, 48)
(28709, 7)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1615, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 30,
        shear_range = 0.3,
        zoom_range = 0.3,
        width_shift_range = 0.4,
        horizontal_flip = True,
        fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = tf.keras.preprocessing.image.NumpyArrayIterator(X_train.reshape(-1,48,48,1), y_train.reshape(-1,7), 
                                                                 train_datagen,batch_size=32, shuffle=True)

test_generator = tf.keras.preprocessing.image.NumpyArrayIterator(X_test.reshape(-1,48,48,1), y_test.reshape(-1,7), test_datagen,
                                                              batch_size=32, shuffle=True)

(24072, 48, 48)
(24072, 7)
(4637, 48, 48)
(4637, 7)


In [0]:
network = tf.keras.applications.ResNet50(include_top=True, weights=None, input_shape=(224,224,1), classes=7)
# network.summary()

In [26]:
network.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.001),
              metrics=['accuracy'])

filepath = "/content/drive/My Drive/emotion_data/resnet50.hdf5"

checkpoint = ModelCheckpoint(filepath,
                            monitor='val_accuracy',
                            verbose=1,
                            save_best_only=True,
                            mode='max')

history = network.fit_generator(train_generator, epochs=1, validation_data=test_generator, callbacks=[checkpoint]) 

752/753 [============================>.] - ETA: 0s - loss: 1.6250 - accuracy: 0.3593
Epoch 00001: val_accuracy improved from -inf to 0.41234, saving model to /content/drive/My Drive/emotion_data/resnet50.hdf5
753/753 [==============================] - 40s 53ms/step - loss: 1.6252 - accuracy: 0.3592 - val_loss: 1.5168 - val_accuracy: 0.4123


In [27]:
history2 = network.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[checkpoint])

Epoch 1/30
753/753 [==============================] - ETA: 0s - loss: 1.6173 - accuracy: 0.3629
Epoch 00001: val_accuracy did not improve from 0.41234
753/753 [==============================] - 39s 51ms/step - loss: 1.6173 - accuracy: 0.3629 - val_loss: 1.5476 - val_accuracy: 0.4005
Epoch 2/30
753/753 [==============================] - ETA: 0s - loss: 1.6187 - accuracy: 0.3598
Epoch 00002: val_accuracy did not improve from 0.41234
753/753 [==============================] - 38s 51ms/step - loss: 1.6187 - accuracy: 0.3598 - val_loss: 1.5297 - val_accuracy: 0.4020
Epoch 3/30
753/753 [==============================] - ETA: 0s - loss: 1.6151 - accuracy: 0.3637
Epoch 00003: val_accuracy improved from 0.41234 to 0.41363, saving model to /content/drive/My Drive/emotion_data/resnet50.hdf5
753/753 [==============================] - 40s 53ms/step - loss: 1.6151 - accuracy: 0.3637 - val_loss: 1.5114 - val_accuracy: 0.4136
Epoch 4/30
752/753 [============================>.] - ETA: 0s - loss: 1.6136

In [0]:
history = network.fit_generator(train_generator, epochs=100, validation_data=test_generator, callbacks=[checkpoint])

Epoch 1/100
752/753 [============================>.] - ETA: 0s - loss: 1.5427 - accuracy: 0.3979
Epoch 00001: val_accuracy improved from 0.43735 to 0.43800, saving model to /content/drive/My Drive/emotion_data/resnet50.hdf5
753/753 [==============================] - 38s 51ms/step - loss: 1.5428 - accuracy: 0.3979 - val_loss: 1.4896 - val_accuracy: 0.4380
Epoch 2/100
752/753 [============================>.] - ETA: 0s - loss: 1.5456 - accuracy: 0.3971
Epoch 00002: val_accuracy improved from 0.43800 to 0.44490, saving model to /content/drive/My Drive/emotion_data/resnet50.hdf5
753/753 [==============================] - 39s 51ms/step - loss: 1.5456 - accuracy: 0.3971 - val_loss: 1.4374 - val_accuracy: 0.4449
Epoch 3/100
753/753 [==============================] - ETA: 0s - loss: 1.5405 - accuracy: 0.3974
Epoch 00003: val_accuracy did not improve from 0.44490
753/753 [==============================] - 37s 49ms/step - loss: 1.5405 - accuracy: 0.3974 - val_loss: 1.5153 - val_accuracy: 0.4182
E